# Dependencies

In [1]:
pacman::p_load(dplyr)

# Functions

In [2]:
source("/Users/anabbi/git/ped_CapTCRseq/R/ggplot2_theme.R")
source("/Users/anabbi/git/ped_CapTCRseq/R/color_schemes.R")

# Paths

In [3]:
datapath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Data/"
plotpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Plots/"
manifestpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Manifests/"
gitpath <- "/Users/anabbi/git/ped_CapTCRseq/"

In [4]:
h4hpath <- "/Users/anabbi/Desktop/H4H/INTERCEPT/"

# Main

bind all metadata

In [5]:
load(file = paste0(datapath,"capTCRseq/meta_div_TRB.RData"))

Warning message in load(file = paste0(datapath, "capTCRseq/meta_div_TRB.RData")):
"strings not representable in native encoding will be translated to UTF-8"


In [6]:
load(file = paste0(datapath, "Adaptivedatasets/allAdaptive.RData"))

In [7]:
# remove those with no TCR data
meta_div <- meta_div[ !is.na(meta_div$file),]

In [8]:
meta_div$study <- "INTERCEPT"
meta_div$Agegroup <- "Child"
meta_div$Sex <- NA
meta_div$disease <- meta_div$cancergroup
meta_div$sample_name <- meta_div$basename

In [9]:
allAdaptive$sampletype <- "PBMC"
allAdaptive$sampletype[grepl("Cord", allAdaptive$sample_name )] <- "Cord blood"

In [10]:
# replace age with Age in allAdative
colnames(allAdaptive)[colnames(allAdaptive) == "age"] <- "Age"
# subject in allAdaptive is the same as Patient in meta_div
colnames(allAdaptive)[colnames(allAdaptive) == "subject"] <- "Patient"

In [11]:
mycols <- intersect(colnames(meta_div), colnames(allAdaptive))

In [12]:
mycols

[1] "Patient"     "Age"         "sampletype"  "study"       "Agegroup"   
[6] "Sex"         "disease"     "sample_name"

In [13]:
all_metadata <- rbind(allAdaptive[,mycols], meta_div[,mycols])

In [14]:
dim(all_metadata)

[1] 1802    8

In [15]:
head(all_metadata)

,Patient,Age,sampletype,study,Agegroup,Sex,disease,sample_name
,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,Preterm_23_3,0,PBMC,Carey,Neonate,Male,Ctrl,Preterm_23_3
2,Preterm_25_5,0,PBMC,Carey,Neonate,Female,Ctrl,Preterm_25_5
3,Preterm_26_B_2,0,PBMC,Carey,Neonate,Male,Ctrl,Preterm_26_B_2
4,Preterm_27_4,0,PBMC,Carey,Neonate,Female,Ctrl,Preterm_27_4
5,Adult_1_15-04,39,PBMC,Carey,Adult,NA,Ctrl,Adult_1_15-04
6,Adult_2_15-07,39,PBMC,Carey,Adult,NA,Ctrl,Adult_2_15-07


In [16]:
table(all_metadata$study)


          Carey         Emerson       Henderson       INTERCEPT        Mitchell 
             19             786              62             576             216 
Mitchell_Denver 
            143 

In [17]:
#save all_metadata
save(all_metadata, file = paste0(datapath, "all_metadata.RData"))

## All PBMC data

In [18]:
# remove cfDNA samples
all_metadata_pbmc <- all_metadata[!grepl("cfDNA", all_metadata$sample_name),]

all_metadata_pbmc <- all_metadata_pbmc[!grepl("FFPE", all_metadata_pbmc$sample_name),]

In [19]:
dim(all_metadata_pbmc)

[1] 1507    8

In [20]:
table(all_metadata_pbmc$study)


          Carey         Emerson       Henderson       INTERCEPT        Mitchell 
             19             786              62             281             216 
Mitchell_Denver 
            143 

## Discovery set

In [38]:
discovery_metadata <- all_metadata_pbmc[ all_metadata_pbmc$study != "Mitchell_Denver",]

In [39]:
discovery_metadata <- discovery_metadata[ !grepl("Keck", discovery_metadata$sample_name),] # emerson validation set

In [40]:
#remove carey Cord samples
discovery_metadata <- discovery_metadata[ !grepl("Cord", discovery_metadata$sample_name),]

In [41]:
dim(discovery_metadata[ discovery_metadata$study == "INTERCEPT",])

[1] 281   8

In [42]:
dim(discovery_metadata)

[1] 1239    8

#### sample intercept study

In [43]:
# subset to INTERCEPT study
tmp <- discovery_metadata[ discovery_metadata$study == "INTERCEPT",]

In [44]:
length(table(tmp$Patient)) # number of patients

[1] 113

In [45]:
myfreq <- as.data.frame(table(tmp$Patient, tmp$disease), stringsAsFactors = F)

In [46]:
#remove zero
myfreq <- myfreq[myfreq$Freq > 0,]

In [47]:
head(myfreq)

,Var1,Var2,Freq
,<chr>,<chr>,<int>
104,LFS_TCR_4535,LFS,1
105,LFS_TCR_4570,LFS,1
106,LFS_TCR_4602,LFS,1
107,LFS_TCR_4758,LFS,1
108,LFS_TCR_4759,LFS,1
109,LFS_TCR_4783,LFS,1


In [48]:
# randomly select 50% of patients from each disease group regardless of how many samples each patient has
set.seed(7)
myfreq %>% group_by(Var2) %>% sample_frac(0.5) -> discovery_sample_freq

In [49]:
discovery_sample_freq

Var1,Var2,Freq
<chr>,<chr>,<int>
LFS_TCR_5067,LFS,1
LFS_TCR_4602,LFS,1
LFS_TCR_4837,LFS,1
LFS_TCR_4758,LFS,1
LFS_TCR_4570,LFS,1
CHP_353,Leukemia,5
CHP_376,Leukemia,2
CHP_339,Leukemia,2
CHP_328,Leukemia,2


In [50]:
# all patients in discovery set
int_discovery_patients <- discovery_sample_freq$Var1

In [51]:
length(int_discovery_patients)

[1] 57

In [52]:
dim(discovery_metadata)

[1] 1239    8

In [53]:
dim(discovery_metadata[ discovery_metadata$study == "INTERCEPT",])

[1] 281   8

In [54]:
all_discovery_patients <- c(discovery_metadata$Patient[ discovery_metadata$study != "INTERCEPT"], int_discovery_patients)

In [55]:
discovery_metadata <- discovery_metadata[ discovery_metadata$Patient %in% all_discovery_patients,]

In [56]:
length(table(discovery_metadata$Patient[ discovery_metadata$study == "INTERCEPT"] ))

[1] 57

In [57]:
# CHP_407 is dupliucate sample one from lymphoma one from ffpe
dim(discovery_metadata)

[1] 1098    8

In [59]:
table(discovery_metadata$study)


    Carey   Emerson Henderson INTERCEPT  Mitchell 
       14       666        62       140       216 

## Validation set

In [60]:
head(all_metadata_pbmc)

,Patient,Age,sampletype,study,Agegroup,Sex,disease,sample_name
,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,Preterm_23_3,0,PBMC,Carey,Neonate,Male,Ctrl,Preterm_23_3
2,Preterm_25_5,0,PBMC,Carey,Neonate,Female,Ctrl,Preterm_25_5
3,Preterm_26_B_2,0,PBMC,Carey,Neonate,Male,Ctrl,Preterm_26_B_2
4,Preterm_27_4,0,PBMC,Carey,Neonate,Female,Ctrl,Preterm_27_4
5,Adult_1_15-04,39,PBMC,Carey,Adult,NA,Ctrl,Adult_1_15-04
6,Adult_2_15-07,39,PBMC,Carey,Adult,NA,Ctrl,Adult_2_15-07


In [65]:
validation_metadata <- all_metadata_pbmc[ !all_metadata_pbmc$Patient %in% all_discovery_patients,]

In [66]:
dim(validation_metadata)

[1] 409   8

In [67]:
dim(discovery_metadata)

[1] 1098    8

In [70]:
# save discovery and validation metadata
save(discovery_metadata, file = paste0(datapath, "discovery_metadata.RData"))
save(validation_metadata, file = paste0(datapath, "validation_metadata.RData"))